In [5]:
import pysam, re, os
from Bio import SeqIO, Seq
import numpy as np

In [6]:
bam_filename = '/home/rcarter/WORK_WEBBY_CHRISTY2/JUPYTER/pre_deadapter/880570-FMT1_S18/tmpEEypDI.bam'
fasta_filename = '/home/rcarter/WORK_WEBBY_CHRISTY2/JUPYTER/pre_deadapter/880570-FMT1_S18/tmp6icc15.fa'
min_coverage = 25


In [7]:
bam_obj = pysam.Samfile(bam_filename, 'rb')

In [8]:
ref_name_to_length_dict = dict(zip(bam_obj.references, bam_obj.lengths))

seg_to_seq_string_dict = {}
for _seqrec in SeqIO.parse(fasta_filename, "fasta"):
    seg_to_seq_string_dict[_seqrec.id] = str(_seqrec.seq)

In [9]:
ref_name_to_length_dict

{'HA': 1795,
 'M1': 1147,
 'NB_NA': 1512,
 'NP': 1750,
 'NS': 1066,
 'PA': 2245,
 'PB1': 2334,
 'PB2': 2396}

In [10]:
counts = bam_obj.count_coverage(reference = 'HA', start = 0, end = ref_name_to_length_dict['HA'])

In [11]:
#foo = counts[0]
tolower_inds_list = []
cov_array = np.asarray(counts)
for _i in range(0,cov_array.shape[1]):
    if(sum(cov_array[:,_i]) < min_coverage):
        tolower_inds_list.append(_i)
seq_base_array = list(seg_to_seq_string_dict['HA'])
for _ind in tolower_inds_list:
    seq_base_array[_ind] = seq_base_array[_ind].lower()


In [50]:
Seq.Seq("".join(seq_base_array))

Seq('cacaaaatgaaggcaataattgtactactcatggtagtaacatccaatgcagat...TTG', Alphabet())

In [29]:
seg_to_seq_string_dict['HA'][4] = 'B'

TypeError: 'str' object does not support item assignment

In [19]:
get_lowercase_seq_coverage(bam_filename, fasta_filename, 10)

[SeqRecord(seq=Seq('gatttgtttagtcactggcaaacagggaaaaatggcgaacaacaacatgaccac...aaa', Alphabet()), id='NS', name='<unknown name>', description='<unknown description>', dbxrefs=[]),
 SeqRecord(seq=Seq('agatgaatataaatccttattttctcttcatagatgtgcccgtacaggcagcaa...aaa', Alphabet()), id='PB1', name='<unknown name>', description='<unknown description>', dbxrefs=[]),
 SeqRecord(seq=Seq('agcagaagcggagcgttttcaagatgacattggccaaaattgaattgttaaaac...act', Alphabet()), id='PB2', name='<unknown name>', description='<unknown description>', dbxrefs=[]),
 SeqRecord(seq=Seq('atggatacttttattacaagaaacttccagactacaataatacaaaaggccaaa...aga', Alphabet()), id='PA', name='<unknown name>', description='<unknown description>', dbxrefs=[]),
 SeqRecord(seq=Seq('aaatgtcgctgtttggagacacaattgcctacctgctttcattgacagaagatg...GGA', Alphabet()), id='M1', name='<unknown name>', description='<unknown description>', dbxrefs=[]),
 SeqRecord(seq=Seq('tcaaaactgaagcaaataggccaaaaatgaacaatgctaccttcaactatacaa...gtt', Alphabet()), id='NB_NA', 

In [18]:
def get_lowercase_seq_coverage(bam_filename, fasta_filename, min_coverage = None):
    '''
    This function reads a bam_filename bam and the corresponding fasta_filename fasta file.
    It converts the sequences from the FASTA file to a combination of uppercase and lowercase characters.
    Lowercase characters are placed wherever the coverage of the corresponding base is less than min_coverage.
    Uppercase characters are placed at all other character positions.

    INPUT:

    bam_filename: a bam filename that will be used to extract all variable
    positions

    fasta_filename: a fasta filename containing the reference sequences from
    bam_filename. This has to be included because bam files do not store the
    reference sequences.

    min_coverage: the minimum coverage of a position  in order for it to be considered covered.
    if None, then all positions are considered covered

    RETURNS:

    a dict mapping each sequence name to a seqrecord of the corresponding sequence.
    The Seq object in the SeqRecord is converted to the upper/lowercase.
    '''

    if not os.path.exists(bam_filename):
        sys.exit("Bam file {} does not exist".format(bam_filename))
    if not os.path.exists(fasta_filename):
        sys.exit("Fasta file {} does not exist".format(fasta_filename))
    try:
        bam_obj = pysam.Samfile(bam_filename, 'rb')
    except Exception:
        sys.exit("Could not load bam file {}".format(bam_filename))

    ref_name_to_length_dict = dict(zip(bam_obj.references, bam_obj.lengths))

    seg_to_seq_string_dict = {}
    try:
        for _seqrec in SeqIO.parse(fasta_filename, "fasta"):
            seg_to_seq_string_dict[_seqrec.id] = str(_seqrec.seq)
    except Exception:
        sys.exit("Could not parse " + fasta_filename)

    seqrec_list = []
    for _refname in seg_to_seq_string_dict.keys():
        tolower_inds_list = []
        counts = bam_obj.count_coverage(reference = _refname, start = 0, end = ref_name_to_length_dict[_refname])
        cov_array = np.asarray(counts)
        for _i in range(0,cov_array.shape[1]):
            if(sum(cov_array[:,_i]) < min_coverage):
                tolower_inds_list.append(_i)
                seq_base_array = list(seg_to_seq_string_dict[_refname])
                for _ind in tolower_inds_list:
                    seq_base_array[_ind] = seq_base_array[_ind].lower()
        seqrec_list.append(SeqRecord.SeqRecord(seq = Seq.Seq("".join(seq_base_array)), id = _refname))
    return seqrec_list
